In [2]:
# Libraries
import collections, re
import numpy as np
import pandas as pd
import treeUtil
import cPickle
from math import log
from nltk.corpus import stopwords

In [28]:
[lib, con, neutral] = cPickle.load(open('ibcData.pkl', 'rb'))

In [29]:
print('Liberal:', len(lib))
print('Neutral:', len(neutral))
print('Conservative:', len(con))

('Liberal:', 2025)
('Neutral:', 600)
('Conservative:', 1701)


In [60]:
## with sub-phrases
# Aggregate into one dataset -- dat
words1 = []
label1 = []
for i, tree in enumerate(lib):
    lst = []
    lst.append(tree)
    leaves = tree.get_leaves()
    while not lst == []:
        curr_node = lst.pop()
        if curr_node and curr_node not in leaves:
            lst.append(curr_node.c1) # get the children, and add them to lst
            lst.append(curr_node.c2) # get the children, and add them to lst
            words1.append(curr_node.get_words())
            label1.append(0)

# n2 = len(neutral)
words2 = []
label2 = []
for i, tree in enumerate(neutral):
    lst = []
    lst.append(tree)
    leaves = tree.get_leaves()
    while not lst == []:
        curr_node = lst.pop()
        if curr_node and curr_node not in leaves:
            lst.append(curr_node.c1) # get the children, and add them to lst
            lst.append(curr_node.c2) # get the children, and add them to lst
            words2.append(curr_node.get_words())
            label2.append(1)
            
words3 = []
label3 = []
for i, tree in enumerate(con):
    lst = []
    lst.append(tree)
    leaves = tree.get_leaves()
    while not lst == []:
        curr_node = lst.pop()
        if curr_node and curr_node not in leaves:
            lst.append(curr_node.c1) # get the children, and add them to lst
            lst.append(curr_node.c2) # get the children, and add them to lst
            words3.append(curr_node.get_words())
            label3.append(2)
            


    
lib_df2 = pd.DataFrame(data = {'words':words1, 'label':label1})
neut_df2 = pd.DataFrame(data = {'words':words2, 'label':label2})
con_df2 = pd.DataFrame(data = {'words':words3, 'label':label3})
frames2 = [lib_df2, con_df2, neut_df2]
dat2 = pd.concat(frames2)

In [30]:
# Aggregate into one dataset -- dat
n1 = len(lib)
words1 = np.full(n1, None)
label1 = np.full(n1, 0)
for i, tree in enumerate(lib):
    words1[i] = tree.get_words()

n2 = len(neutral)
words2 = np.full(n2, None)
label2 = np.full(n2, 1)
for i, tree in enumerate(neutral):
    words2[i] = tree.get_words()
       
n3 = len(con)
words3 = np.full(n3, None)
label3 = np.full(n3, 2)
for i, tree in enumerate(con):
    words3[i] = tree.get_words()

    
lib_df = pd.DataFrame(data = {'words':words1, 'label':label1})
neut_df = pd.DataFrame(data = {'words':words2, 'label':label2})
con_df = pd.DataFrame(data = {'words':words3, 'label':label3})
frames = [lib_df, con_df, neut_df]
dat = pd.concat(frames)

In [31]:
# Function to clean word data--removes stopwords, makes lowercase, removes numbers
def clean_words(sentences):
    words_clean = np.full(len(sentences), None)
    for i, words in enumerate(sentences):
        word_list = re.split('\W+', words)
        words1 = [word.lower() for word in word_list if word.lower() not in stopwords.words('english')]
        words2 = [word for word in words1 if not any(char.isdigit() for char in word)]
        words_clean[i] = (' '.join(words2)).strip()
    return words_clean

In [32]:
dat['words_clean'] = clean_words(dat['words'])

In [61]:
dat2['words_clean'] = clean_words(dat2['words'])

In [33]:
dat['words_clean'].head(n=10)

0    forcing middle class workers bear greater shar...
1    would worthwhile bring case arbitration clause...
2    indeed lind argues high profits high wages rei...
3    fairness noted devotes entire chapter new york...
4    psychological tactics social control technique...
5    uncontrolled profit motive destroying health i...
6    organizations representing religious right loy...
7    market based greed fear tugged worst things us...
8    conservative movement rooted coherent easy sum...
9    eliminating private insurer could save billion...
Name: words_clean, dtype: object

In [34]:
# Look at words
i = 0
print(dat['words'].iloc[i])
print(dat['words_clean'].iloc[i])
print(dat['label'].iloc[i])

Forcing middle-class workers to bear a greater share of the cost of government weakens their support for needed investments and stirs resentment toward those who depend on public services the most .
forcing middle class workers bear greater share cost government weakens support needed investments stirs resentment toward depend public services
0


In [35]:
dat.head()

,label,words,words_clean
0,0,Forcing middle-class workers to bear a greater...,forcing middle class workers bear greater shar...
1,0,Because it would not be worthwhile to bring a ...,would worthwhile bring case arbitration clause...
2,0,"Indeed , Lind argues that high profits and hig...",indeed lind argues high profits high wages rei...
3,0,"In fairness , it should be noted that he devot...",fairness noted devotes entire chapter new york...
4,0,Psychological tactics are social control techn...,psychological tactics social control technique...


In [36]:
dat.to_excel('ibc.xlsx')

In [62]:
dat2.to_excel('ibc3.xlsx')

In [37]:
new_dat = pd.read_excel('additional_sentences.xlsx')

In [38]:
print(new_dat.shape)
new_dat.head()

(1152, 2)


,sentences,score
3.0,thats all there was to it.,1
66.0,these are the gifts that can impact their lives.,1
17.0,"representative joe barton of texas, the states...",0
32.0,"""i believe his position has to do whether we f...",1
12.0,please try again later.,1


In [39]:
manual = pd.DataFrame(columns=['label','words','words_clean'])
manual['label'] = new_dat['score']
manual['words'] = new_dat['sentences']
manual['words_clean'] = clean_words(new_dat['sentences'])

In [40]:
print(manual.shape)
manual.head()

(1152, 3)


,label,words,words_clean
3.0,1,thats all there was to it.,thats
66.0,1,these are the gifts that can impact their lives.,gifts impact lives
17.0,0,"representative joe barton of texas, the states...",representative joe barton texas states senior ...
32.0,1,"""i believe his position has to do whether we f...",believe position whether follow constitution r...
12.0,1,please try again later.,please try later


In [41]:
frames = [dat, manual]
dat = pd.concat(frames)

In [42]:
print(dat.shape)
dat.head()

(5478, 3)


,label,words,words_clean
0.0,0,Forcing middle-class workers to bear a greater...,forcing middle class workers bear greater shar...
1.0,0,Because it would not be worthwhile to bring a ...,would worthwhile bring case arbitration clause...
2.0,0,"Indeed , Lind argues that high profits and hig...",indeed lind argues high profits high wages rei...
3.0,0,"In fairness , it should be noted that he devot...",fairness noted devotes entire chapter new york...
4.0,0,Psychological tactics are social control techn...,psychological tactics social control technique...


In [43]:
dat = dat[dat['words_clean'].map(len) > 15]

In [44]:
print(dat.shape)
dat.head()

(5372, 3)


,label,words,words_clean
0.0,0,Forcing middle-class workers to bear a greater...,forcing middle class workers bear greater shar...
1.0,0,Because it would not be worthwhile to bring a ...,would worthwhile bring case arbitration clause...
2.0,0,"Indeed , Lind argues that high profits and hig...",indeed lind argues high profits high wages rei...
3.0,0,"In fairness , it should be noted that he devot...",fairness noted devotes entire chapter new york...
4.0,0,Psychological tactics are social control techn...,psychological tactics social control technique...


In [45]:
dat.to_excel('all_sentences.xlsx')